In [113]:
import numpy as np

Enunciado exercício class: <br>
Peça para o usuário uma lista com as comidas favoritas dele, depois transforme isso em um objeto numpy. Agora transformado, elimine todas as comidas que começam com uma vogal.

In [114]:
comidas = np.array(['maçã', 'uva', 'morango', 'melancia', 'acelga'])
comidas

array(['maçã', 'uva', 'morango', 'melancia', 'acelga'], dtype='<U8')

In [115]:
[item[0] for item in comidas ]

['m', 'u', 'm', 'm', 'a']

In [116]:
comidas[np.char.endswith(comidas, 'a')]

array(['uva', 'melancia', 'acelga'], dtype='<U8')

In [117]:
[item for item in comidas if item[0] not in ['a','e','i','o','u']]

['maçã', 'morango', 'melancia']

In [118]:
prefixes = ["xyz", "abc"]
"abcde".startswith(tuple(prefixes))

True

In [119]:
prefixes = ["a", "e", "i"]

[item for item in comidas if not item.startswith(tuple(prefixes))]

['maçã', 'uva', 'morango', 'melancia']

# Aula 3 - Limpeza e transformação de dados

Na aula de hoje, vamos explorar os seguintes tópicos em Python:

- 1) Limpeza de Dados (dropna, fillna, isnull, notnull, replace, duplicated, drop_duplicated)
- 2) Transformação de Dados (cut, qcut, get_dummies)

________

### Objetivos

Apresentar como identificamos valores nulos, discutir melhores formas de tratá-los além de trabalhar com variáveis dummies.

________

### Habilidades a serem desenvolvidas nessa aula

Ao final da aula o aluno deve:

- Saber como identificar valores nulos e como tratá-los.
- Identificar quando utilizar variáveis dummies e como criá-las.
____
____
____

In [120]:
import pandas as pd 
import numpy as np

In [121]:
df = pd.read_csv("data/titanic.csv")

## Limpeza de Dados: Dados Faltantes

### `.isna() e isnull()`
retorna um df booleano indicando se existe um nulo naquela posição. `pd.isnull()` é um alias para `pd.isna()` dentro do python como podemos ver na <a href="https://github.com/pandas-dev/pandas/blob/0409521665bd436a10aea7e06336066bf07ff057/pandas/core/dtypes/missing.py#L109">documentação</a>. O pandas dataframe é baseado nos df do R, onde null e na tem sentidos distintos.

In [122]:
pd.isnull

<function pandas.core.dtypes.missing.isna(obj)>

Tanto o `.isna()` quanto o `.isnull()` retornam um df booleano:

In [123]:
df.isna()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,False,False,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,True,False
3,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
886,False,False,False,False,False,False,False,False,False,False,True,False
887,False,False,False,False,False,False,False,False,False,False,False,False
888,False,False,False,False,False,True,False,False,False,False,True,False
889,False,False,False,False,False,False,False,False,False,False,False,False


Para termos a quantidade de nans em cada coluna podemos somar os valores true:

In [124]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

### `.notnull()`

retorna df com True ou False, com True para elementos não nulos

In [125]:
df.notnull().sum()

PassengerId    891
Survived       891
Pclass         891
Name           891
Sex            891
Age            714
SibSp          891
Parch          891
Ticket         891
Fare           891
Cabin          204
Embarked       889
dtype: int64

### `.dropna()`

elimina nulos em todo o df ou em colunas especificadas quando utilizado o argumento `subset`. É um dos métodos que aceita o parâmetro `inplace`.


In [126]:
df.shape

(891, 12)

In [127]:
df.dropna(subset=['Age','Embarked']).shape

(712, 12)

In [128]:
df.dropna(subset=['Age','Embarked']).isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          529
Embarked         0
dtype: int64

### `.fillna()`

preeche os valores nulos com valores especificados.
Parâmetros úteis:

   * method {‘bfill’,‘ffill’, None}, default None <br>
       Preenche os nas propagando o último valor válido para frente (ffill) ou utiliza próxima observação válida para preencher os nulos (bfill).
   * axis {0 para ‘index’, 1 para ‘colunas’} <br>
       Eixo no qual se quer preencher os nulos
   * inplace {booleano}, default False


In [129]:
fill = pd.DataFrame({'original': [np.nan,1,2,np.nan,4,5,np.nan,np.nan, np.nan, 9, 10, np.nan]})
fill

,original
0,NaN
1,1.0
2,2.0
3,NaN
4,4.0
5,5.0
6,NaN
7,NaN
8,NaN
9,9.0


In [130]:
fill.original.mean()

5.166666666666667

In [131]:
fill['cte'] = fill.original.fillna(999)
fill['media'] = fill.original.fillna(fill.original.mean())
fill['ffill'] = fill.original.fillna(method='ffill')
fill['ffill_limit'] = fill.original.fillna(method='ffill', limit=2)
fill['bfill'] = fill.original.fillna(method='bfill')

fill

,original,cte,media,ffill,ffill_limit,bfill
0,NaN,999.0,5.166667,NaN,NaN,1.0
1,1.0,1.0,1.000000,1.0,1.0,1.0
2,2.0,2.0,2.000000,2.0,2.0,2.0
3,NaN,999.0,5.166667,2.0,2.0,4.0
4,4.0,4.0,4.000000,4.0,4.0,4.0
5,5.0,5.0,5.000000,5.0,5.0,5.0
6,NaN,999.0,5.166667,5.0,5.0,9.0
7,NaN,999.0,5.166667,5.0,5.0,9.0
8,NaN,999.0,5.166667,5.0,NaN,9.0
9,9.0,9.0,9.000000,9.0,9.0,9.0


### Voltando ao dataset do titanic...
Quais soluções poderíamos utilizar para preencher os nulos em Age? <br>
E as cabines nulas? <br>
E os portões de embarque nulos? <br>
Qual os possíveis problemas que podemos encontrar utilizando essas abordagens? <br>


_________
_________
**Exercício:** Ordene as colunas Pclass e Fare, nessa ordem e ascendente, e preencha os nulos utilizando o método foward fill.

In [132]:
df['new_cabin'] = df.sort_values(["Pclass", "Fare"])['Cabin'].fillna(method="ffill")

_________
_________

### `.interpolate()`
Faz a interpolação dos missing de acordo com o método escolhido.
[Documentação](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.interpolate.html)

In [133]:
s = pd.DataFrame([0, 1, np.nan, 3, 4, np.nan, 400], columns=['original'])
s['linear'] = s.original.interpolate()
s['segundo_grau'] = s.original.interpolate(method="polynomial", order=2)
s

ImportError: Missing optional dependency 'scipy'. polynomial interpolation requires SciPy. Use pip or conda to install scipy.

### `.replace()`

substitui elementos dentro do df. É um dos métodos que aceita o parâmetro `inplace`.


In [134]:
# para apenas uma troca
df.replace({'h':'male'}, inplace=True)

In [135]:
df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,new_cabin
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,F4
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,C85
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,F38
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,C123
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,F38
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,E10
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,E46
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,G6
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,G6
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,F2


In [136]:
df.Sex.replace({'male':'H', 'female':'M'})

0      H
1      M
2      M
3      M
4      H
      ..
886    H
887    M
888    M
889    H
890    H
Name: Sex, Length: 891, dtype: object

In [137]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,new_cabin
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,F4
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,C85
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,F38
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,C123
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,F38


In [138]:
# para mais de uma troca
df.replace({'Sex':{'male':'H', 'female':'M'}, 'Name':{'Braund, Mr. Owen Harris':'fera'}})

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,new_cabin
0,1,0,3,fera,H,22.0,1,0,A/5 21171,7.2500,NaN,S,F4
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",M,38.0,1,0,PC 17599,71.2833,C85,C,C85
2,3,1,3,"Heikkinen, Miss. Laina",M,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,F38
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",M,35.0,1,0,113803,53.1000,C123,S,C123
4,5,0,3,"Allen, Mr. William Henry",H,35.0,0,0,373450,8.0500,NaN,S,F38
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",H,27.0,0,0,211536,13.0000,NaN,S,F33
887,888,1,1,"Graham, Miss. Margaret Edith",M,19.0,0,0,112053,30.0000,B42,S,B42
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",M,NaN,1,2,W./C. 6607,23.4500,NaN,S,F E69
889,890,1,1,"Behr, Mr. Karl Howell",H,26.0,0,0,111369,30.0000,C148,C,C148


___________________
___________________

**Exercício**: Troque os valores abreviados do portão de embarque para seus respectivos nomes 
(C = Cherbourg; Q = Queenstown; S = Southampton) e preencha os dados faltantes com o portão mais frequente

In [139]:
# Resposta
20:14

SyntaxError: illegal target for annotation (Temp/ipykernel_17856/3838409541.py, line 2)

In [140]:
df.Embarked = df.replace({'Embarked': 
            {'C': 'Cherbourg', 'Q':'Queenstown', 'S':'Southampton'}
           })['Embarked'].fillna('mode')

In [141]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,new_cabin
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,Southampton,F4
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,Cherbourg,C85
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,Southampton,F38
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,Southampton,C123
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,Southampton,F38


In [142]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
new_cabin        0
dtype: int64

___________________
___________________


## Limpeza de Dados: Dados duplicados
### `.duplicated()` e `.drop_duplicated()`

O `.duplicated()` retorna uma series indicando se determinada linha possui duplicados ou não. Já o `.drop_duplicated()`, elimina as linhas duplicadas. <br>
Parâmetros do `.drop_duplicated()`:

   * `subset` -  seleciona colunas para serem utilizadas na comparação de linhas duplicadas
   * `keep` - {‘first’, ‘last’, False} (default ‘first’) Determina qual duplicado manter
   * `inplace` - (default False) Se False retorna uma cópia do df com as alterações, se True faz as modificações no próprio df



In [143]:
# vamos criar um df que possua linhas duplicadas
df_dup = df.copy()

df_dup.loc[891, :] = df_dup.loc[0, :]
df_dup.loc[892, :] = df_dup.loc[1, :]
df_dup.loc[893, :] = df_dup.loc[1, :]

print(df.shape, df_dup.shape)

(891, 13) (894, 13)


In [144]:
df_dup

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,new_cabin
0,1.0,0.0,3.0,"Braund, Mr. Owen Harris",male,22.0,1.0,0.0,A/5 21171,7.2500,NaN,Southampton,F4
1,2.0,1.0,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1.0,0.0,PC 17599,71.2833,C85,Cherbourg,C85
2,3.0,1.0,3.0,"Heikkinen, Miss. Laina",female,26.0,0.0,0.0,STON/O2. 3101282,7.9250,NaN,Southampton,F38
3,4.0,1.0,1.0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1.0,0.0,113803,53.1000,C123,Southampton,C123
4,5.0,0.0,3.0,"Allen, Mr. William Henry",male,35.0,0.0,0.0,373450,8.0500,NaN,Southampton,F38
...,...,...,...,...,...,...,...,...,...,...,...,...,...
889,890.0,1.0,1.0,"Behr, Mr. Karl Howell",male,26.0,0.0,0.0,111369,30.0000,C148,Cherbourg,C148
890,891.0,0.0,3.0,"Dooley, Mr. Patrick",male,32.0,0.0,0.0,370376,7.7500,NaN,Queenstown,F38
891,1.0,0.0,3.0,"Braund, Mr. Owen Harris",male,22.0,1.0,0.0,A/5 21171,7.2500,NaN,Southampton,F4
892,2.0,1.0,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1.0,0.0,PC 17599,71.2833,C85,Cherbourg,C85


In [145]:
df_dup.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
889    False
890    False
891     True
892     True
893     True
Length: 894, dtype: bool

In [146]:
df_dup.drop_duplicates(subset=['Age', 'Embarked'])

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,new_cabin
0,1.0,0.0,3.0,"Braund, Mr. Owen Harris",male,22.0,1.0,0.0,A/5 21171,7.2500,NaN,Southampton,F4
1,2.0,1.0,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1.0,0.0,PC 17599,71.2833,C85,Cherbourg,C85
2,3.0,1.0,3.0,"Heikkinen, Miss. Laina",female,26.0,0.0,0.0,STON/O2. 3101282,7.9250,NaN,Southampton,F38
3,4.0,1.0,1.0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1.0,0.0,113803,53.1000,C123,Southampton,C123
5,6.0,0.0,3.0,"Moran, Mr. James",male,NaN,0.0,0.0,330877,8.4583,NaN,Queenstown,E10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
829,830.0,1.0,1.0,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0.0,0.0,113572,80.0000,B28,mode,B28
843,844.0,0.0,3.0,"Lemberopolous, Mr. Peter L",male,34.5,0.0,0.0,2683,6.4375,NaN,Cherbourg,F4
851,852.0,0.0,3.0,"Svensson, Mr. Johan",male,74.0,0.0,0.0,347060,7.7750,NaN,Southampton,F38
852,853.0,0.0,3.0,"Boulos, Miss. Nourelain",female,9.0,1.0,1.0,2678,15.2458,NaN,Cherbourg,E121


In [147]:
print(df_dup.shape)
print(df_dup.drop_duplicates().shape)

(894, 13)
(891, 13)


In [148]:
%%timeit
df_dup.drop_duplicates(subset=['PassengerId'])

1.96 ms ± 461 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [149]:
%%timeit
df_dup.sort_values('PassengerId').drop_duplicates(subset=['PassengerId'])

3 ms ± 549 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


_______________________
_______________________
**Exercícios:** Elimine os duplicados considerando as colunas 'brand' e 'style' mantendo o último valor.

In [150]:
df_exerc = pd.DataFrame({
    'brand': ['Yum Yum', 'Yum Yum', 'Yum Yum', 'Indomie', 'Indomie', 'Indomie'],
    'style': ['cup', 'cup', 'cup', 'cup', 'pack', 'pack'],
    'rating': [4, 4, np.nan, 3.5, 15, 5]
})

In [151]:
df_exerc

,brand,style,rating
0,Yum Yum,cup,4.0
1,Yum Yum,cup,4.0
2,Yum Yum,cup,NaN
3,Indomie,cup,3.5
4,Indomie,pack,15.0
5,Indomie,pack,5.0


In [152]:
df_exerc.drop_duplicates(subset=['brand','style'], inplace=True, keep='last')

In [153]:
df_exerc

,brand,style,rating
2,Yum Yum,cup,NaN
3,Indomie,cup,3.5
5,Indomie,pack,5.0


__________
__________


## Transformação de dados

### `.get_dummies()`

#### variáveis categóricas
Variáveis categóricas são aquelas que representam grupos ou classes dentro dos nossos dados. Elas podem ser de dois tipos:
* ordinais: possuem uma ordem que tem um sentido. Por exemplo, em rendimentos poderíamos ter: classe alta > classe média > classe baixa  
* nominais: não possuem uma ordem válida. Por exemplo: sexo e CEP.

<img src="variaveis_categoricas.jpeg" style="width: 500px">

Dummies são quaisquer variáveis cujos valores são 1 ou 0 para cada observação. O método `pd.get_dummies()` converte as variáveis categóricas em numéricas separando cada categoria em uma coluna única.
<br>
<br>
<a href="https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html">Parâmetros:</a> <br>
`pandas.get_dummies(data, prefix=None, prefix_sep='_', dummy_na=False, columns=None, sparse=False, drop_first=False, dtype=None)`

In [154]:
pd.get_dummies(df, columns=['Sex', 'Embarked'], drop_first=True)

,PassengerId,Survived,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,new_cabin,Sex_male,Embarked_Queenstown,Embarked_Southampton,Embarked_mode
0,1,0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,F4,1,0,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C85,0,0,0,0
2,3,1,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,F38,0,0,1,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,C123,0,0,1,0
4,5,0,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,F38,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",27.0,0,0,211536,13.0000,NaN,F33,1,0,1,0
887,888,1,1,"Graham, Miss. Margaret Edith",19.0,0,0,112053,30.0000,B42,B42,0,0,1,0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",NaN,1,2,W./C. 6607,23.4500,NaN,F E69,0,0,1,0
889,890,1,1,"Behr, Mr. Karl Howell",26.0,0,0,111369,30.0000,C148,C148,1,0,0,0


## Material de Aprofundamento

### `.cut()`
O método `.cut()` ordena os dados, separa em bins e computa qual grupo cada linha do df pertence. O `.cut()` escolherá os bins para serem espaçados uniformemente de acordo com os próprios valores e não com a frequência desses valores.  <br>
Ele é muito utilizado para transformar variáveis contínuas em categóricas. Por exemplo, podemos converter o valor númerico da idade em grupos de criança, jovem, adulto e idoso.
<br><br>
<a href='https://pandas.pydata.org/docs/reference/api/pandas.cut.html'>Parâmetros:</a> <br>
`pandas.cut(x, bins, right=True, labels=None, retbins=False, precision=3, include_lowest=False, duplicates='raise', ordered=True)`

Ao informar quantidade de grupos o pd.cut() escolhe os bins com o mesmo tamanho de janela :


In [155]:
df['cut_bins'] = pd.cut(df.Age, 4)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,new_cabin,cut_bins
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,Southampton,F4,"(20.315, 40.21]"
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,Cherbourg,C85,"(20.315, 40.21]"
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,Southampton,F38,"(20.315, 40.21]"
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,Southampton,C123,"(20.315, 40.21]"
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,Southampton,F38,"(20.315, 40.21]"


In [156]:
df.cut_bins.value_counts()

(20.315, 40.21]    385
(0.34, 20.315]     179
(40.21, 60.105]    128
(60.105, 80.0]      22
Name: cut_bins, dtype: int64

Podemos passar o nome dos grupos e transformar a variável numérica diretamente em categórica


In [157]:
df['cut_classes'] = pd.cut(df.Age, 4, labels=["jovens", "adultos", "meia-idade", "idosos"])
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,new_cabin,cut_bins,cut_classes
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,Southampton,F4,"(20.315, 40.21]",adultos
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,Cherbourg,C85,"(20.315, 40.21]",adultos
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,Southampton,F38,"(20.315, 40.21]",adultos
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,Southampton,C123,"(20.315, 40.21]",adultos
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,Southampton,F38,"(20.315, 40.21]",adultos


In [158]:
df.cut_bins.unique()

[(20.315, 40.21], NaN, (40.21, 60.105], (0.34, 20.315], (60.105, 80.0]]
Categories (4, interval[float64, right]): [(0.34, 20.315] < (20.315, 40.21] < (40.21, 60.105] < (60.105, 80.0]]

In [159]:
df.cut_classes.value_counts()

adultos       385
jovens        179
meia-idade    128
idosos         22
Name: cut_classes, dtype: int64

Também podemos passar uma lista com os valores de início e fim dos bins:

In [160]:
pd.cut(df.Age, [0,20,60,80]).unique()

[(20.0, 60.0], NaN, (0.0, 20.0], (60.0, 80.0]]
Categories (3, interval[int64, right]): [(0, 20] < (20, 60] < (60, 80]]

In [161]:
df.Age.describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

<a href='https://towardsdatascience.com/discretisation-using-decision-trees-21910483fa4b'>Discretização utilizando decision trees</a>

### `.qcut()`
O `pd.qcut()` é utilizado quando queremos discretizar nossos dados em quantis. Ao informar quantidade de grupos o `pd.qcut()` escolhe os bins tal que tenhamos a mesma quantidade de valores em cada grupo.

#### `.cut()` x `.qcut()`
   * O comando `.cut()` cria **caixas equidistantes**, mas a **frequência** das amostras é **desigual** em cada caixa
   * O comando `.qcut()` cria **caixas de tamanhos desiguais**, mas a **frequência** das amostras é **igual** em cada caixa.

<br>

[Parâmetros](https://pandas.pydata.org/docs/reference/api/pandas.qcut.html)
<br>
`pandas.qcut(x, q, labels=None, retbins=False, precision=3, duplicates='raise')`

In [162]:
pd.cut(df.Age, 4).value_counts()

(20.315, 40.21]    385
(0.34, 20.315]     179
(40.21, 60.105]    128
(60.105, 80.0]      22
Name: Age, dtype: int64

In [163]:
pd.qcut(df.Age, 4).value_counts()

(20.125, 28.0]     183
(0.419, 20.125]    179
(38.0, 80.0]       177
(28.0, 38.0]       175
Name: Age, dtype: int64

In [164]:
pd.qcut(df.Age, 4).value_counts()/df.Age.notnull().sum()

(20.125, 28.0]     0.256303
(0.419, 20.125]    0.250700
(38.0, 80.0]       0.247899
(28.0, 38.0]       0.245098
Name: Age, dtype: float64

## Exercícios

1. Converta os países do dataframe abaixo para variáveis dummies eliminando a primeira coluna e utilizando 'Country' como prefixo no nome das colunas.

In [169]:
import pandas as pd
ids = [11, 22, 33, 44, 55, 66, 77]
countries = ['Brasil', 'Argentina', 'Peru', 'Bolívia', 'Uruguai']
sales = [42, 53, 37, 45, 20]

df = pd.DataFrame(list(zip(ids, countries, sales)),
                  columns=['Ids', 'Countries', 'Sales'])
df.head()
pd.get_dummies(df, columns=['Countries'], prefix='Country', drop_first=True)

,Ids,Sales,Country_Bolívia,Country_Brasil,Country_Peru,Country_Uruguai
0,11,42,0,1,0,0
1,22,53,0,0,0,0
2,33,37,0,0,1,0
3,44,45,1,0,0,0
4,55,20,0,0,0,1


2. Considere os dados do arquivo "german_credit.csv" que contem dados de empréstimos realizados por um banco.<br> <br>
a. Tem nulos nesse df? Se sim, preencha os nulos da forma que o grupo decidir.

In [181]:
df = pd.read_csv("./data/german_credit.csv")
df.isna().sum()
# df.dropna()
df['Saving accounts'] = df['Saving accounts'].fillna(df['Saving accounts'].mode())
df['Checking account'] = df['Checking account'].fillna(df['Checking account'].mode())
df

,Unnamed: 0,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose
0,0,67,male,2,own,little,little,1169,6,radio/TV
1,1,22,female,2,own,little,moderate,5951,48,radio/TV
2,2,49,male,1,own,little,NaN,2096,12,education
3,3,45,male,2,free,little,little,7882,42,furniture/equipment
4,4,53,male,2,free,little,little,4870,24,car
...,...,...,...,...,...,...,...,...,...,...
995,995,31,female,1,own,little,NaN,1736,12,furniture/equipment
996,996,40,male,3,own,little,little,3857,30,car
997,997,38,male,2,own,little,NaN,804,12,radio/TV
998,998,23,male,2,free,little,little,1845,45,radio/TV


   b. Converta as variáveis categóricas em numéricas.

3. Para esse exercício vamos considerar o dataset https://archive.ics.uci.edu/ml/datasets/automobile que traz a especificação de modelos de carros em 1985.
https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data

In [112]:
# Considere a seguinte lista como o nome das colunas do dataset
headers = ["symboling", "normalized_losses", "make", "fuel_type", "aspiration",
           "num_doors", "body_style", "drive_wheels", "engine_location",
           "wheel_base", "length", "width", "height", "curb_weight",
           "engine_type", "num_cylinders", "engine_size", "fuel_system",
           "bore", "stroke", "compression_ratio", "horsepower", "peak_rpm",
           "city_mpg", "highway_mpg", "price"]

a. Esse dataset possui nans? Se sim, o grupo precisa entrar em um acordo de como tratá-los.

b. Esse dataset possui linhas duplicadas?

c. O consumo dos automóveis na cidade e rodovia, representados pelas colunas "city-mpg" e "highway-mpg" respectivamente, estão na unidade mpg (miles per gallon). Converta essas colunas para consumo em L/100km sabendo que a conversão é dada por:

L/100km = 235 / mpg 

d. Converta as colunas "body_style" e "drive_wheels" para variáveis dummies.

## Referências:
